# Assignment 3 - SED Fitting

## Bayesian Statistics

This assignment is split into 3 sections, roughly corresponding to the contents of each of the 3 weeks in the Bayesian Statistics module. 

All assignments are presented as Jupyter notebooks. You will fork the repository to have your own access to all files. You can edit this notebook directly with your answers and push your changes to GitHub. 

### **The goal of this assignment is to use different MCMC and Bayesian inference techniques to fit SEDs to galaxy magnitudes**

# STEP 0 - Prospector Inputs

Prospector has some in-built MCMC techniques that you will have used in Assignment 1 (Emcee or Dynesty). For this assignment you will not be using these in-built tools, instead you will use external libraries to code your own MCMC results and perform model comparison on them. We will however still use prospector for the model so lets prepare that here

### The model we are using is a very simple parametric model with 6 free parameters

* ### $z$ redshift
* ### $M_{\rm star}$ stellar mass
* ### $\log(z_{\rm sol})$ metallicity?
* ### $\rm dust$ a dust parameter
* ### $t_{\rm age}$ 
* ### $\tau$ something


1. Activate the enviroment/kernel you used with prospector installed

2. Prepare the prospector model (if you like you can edit the below model, but you dont have to)

In [13]:
from prospect.models import SedModel, priors
from prospect.models.templates import TemplateLibrary
from prospect.sources import CSPSpecBasis
import time

model_params = TemplateLibrary["parametric_sfh"]

# Let redshift vary
model_params["zred"]["isfree"] = True
model_params['zred']['init'] = 0.1
model_params['zred']['prior'] = priors.TopHat(mini=0,maxi=1)

# Build the model
prospector_model = SedModel(model_params)

sps = CSPSpecBasis(zcontinuous=1)


3. Load the data vector for a given galaxy (again you can change the gaalxy if you wish, maybe match one of your galaxies from assignment)

In [14]:
import sedpy 
import prospect
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table

gal_id = 33

with fits.open('../data/sw_input.fits') as f:
    df = Table(f[1].data).to_pandas()
    f.close()

def build_obs(gal_id):
    """Given an object, load in fluxes, convert them to nanomaggies, and create a dict used in Prospector."""

    inp = {}
    
    # Get dataframe row for the object
    row = df.iloc[gal_id]
    inp['redshift'] = row.redshift

    # Load the filter response curves from sedpy
    bands = [f'sdss_{filt}0' for filt in 'ugriz'] + [f'wise_w{n}' for n in range(1,5)]
    filters = sedpy.observate.load_filters(bands)
    inp['filters'] = filters
    
    # Fluxes and uncertainties - already in units of maggies
    cols = [f'flux_{filt}' for filt in 'ugriz'] + [f'flux_w{n}' for n in range(1,5)]
    fluxes = row[cols].values.astype(float) / 3631

    # Errors
    cols_err = [f'{col}_e' for col in cols]
    errs = row[cols_err].values.astype(float) / 3631

    # Anything with a value of 9.999 is null, so may need to mask those fluxes by editing phot_mask
    inp['maggies'] = fluxes
    inp['maggies_unc'] = errs
    inp['phot_mask'] = [True for val in fluxes] # Nothing masked here right now

    # This is an array of effective wavelengths for each of the filters.  
    # It is not necessary, but it can be useful for plotting so we store it here as a convenience
    inp["phot_wave"] = np.array([f.wave_effective for f in inp["filters"]])
    inp["wavelength"] = None
    
    # Populate other fields with default
    inp = prospect.utils.obsutils.fix_obs(inp)
    return inp

obs = build_obs(gal_id)

4. Prepare a function that takes the 6 parameters as input, and outputs the predicted fluxes in maggies (once this is set up, you shouldn't have to use prospector directly again for this assignment)

In [30]:
#a random sample from the prospectors default priors (just to make sure the model runs)
random_input_values = np.array([model_params[k]['prior'].sample()[0] for k in model_params.keys() if model_params[k]['isfree']==True])
param_names = np.array([model_params[k]['name'] for k in model_params.keys() if model_params[k]['isfree']==True])
print('Parameters should enter the function in this order:')
print(param_names)

#the first model takes longer to run so do that here
start = time.time()
test_model1 = prospector_model.predict(random_input_values, obs=obs, sps=sps)
finish = time.time()
print(f'The first model evaluation took {finish-start}s')

#future calls should be quick
### THIS IS THE FUNCTION YOU CAN USE FOR ALL 
### YOUR MODEL PREDICTIONS FROM THIS POINT
def model(theta):
    return prospector_model.predict(theta, obs=obs, sps=sps)

# see how long one model prediction takes
start = time.time()
test_model2 = model(random_input_values*1.2) #change the input params a little here so we know its not just caching outputs
finish = time.time()
print(f'Subsequent evaluations will take {finish-start}s')



Parameters should enter the function in this order:
['zred' 'mass' 'logzsol' 'dust2' 'tage' 'tau']
The first model evaluation took 26.653258085250854s
Subsequent evaluations will take 12.884146213531494s


# STEP 1 - MCMC

1. Pick (and install) your 2 favourite MCMC sampling method/implementation you learned about in the lectures (At least 1 of them should be a "nested" sampling method, as we will use the features of nested sampling later)

Some examples are shown below

Metropolis hastings
* links 

Emsemble
* Affine-invariant [EMCEE](https://emcee.readthedocs.io/en/stable/)
* KDE [Kombine](https://github.com/bfarr/kombine)

Nested sampling 
* Multinest
* Polychord
* [Nautilus](https://nautilus-sampler.readthedocs.io/en/stable/)
* Dynesty

In [16]:
### Space to work

2. Fit your model to your data using your 2 chosen sampling techniques

* Choose flat priors for all parameters (you can look at your results from Assignment 1 to get some reasonable ranges) 
* Plot the resulting parameter constraints on top of each other. e.g. in a corner plot
* Do they agree with each other?

In [17]:
### Space to work

3. Did either of your sampling methods have a burn-in? if so make a plot showing the burn-in and justify how much burn-in to remove. If not, explain why there is no burn-in in your methods

In [18]:
### Space to work

# STEP 2 - Model Comparison

1. Use the nested sampling technique to compute the Baysian Evidence of your model fit

In [19]:
### Space to work

2. Change the model in some-way and re-run your nested sampling chain (e.g. add a new parameter, or dramatically change the prior on a parameter)

In [20]:
### Space to work

3. Use a Bayesian model comparison technique to decide which model your data prefer

* WE SHOULD CHECK THE SLIDES TO SEE WHICH ONES WILL BE COVERED

In [21]:
### Space to work

4. How do the conclusions from your "Bayesian" model comparison compare to just looking at the change in the best-fit (reduced) chi-squared

# STEP 3 - Advanced MCMC techniques



1. 